In [4]:
import pdfplumber
import pymongo
import pandas as pd
from pymongo import MongoClient
import pandas as pd

In [7]:
import pdfplumber
import pandas as pd

def extract_dimensions_with_sizes(pdf_path):
    dimensions = []
    headers = []  # To store the main and sub-headers dynamically
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                if len(table) > 0:
                    # Extract the headers (first row) and data rows
                    current_headers = table[0]
                    if not headers:  # Capture headers only once
                        headers = current_headers
                    for row in table[1:]:
                        if len(row) == len(headers):  # Ensure row matches headers length
                            row_data = dict(zip(headers, row))
                            dimensions.append(row_data)
    return dimensions, headers

def process_dimensions_with_sizes(dimensions, headers):
    processed_data = []
    for record in dimensions:
        processed_record = {}
        for header, value in record.items():
            # Normalize header names
            if header and header.strip():
                processed_record[header.strip()] = value
        processed_data.append(processed_record)

    # Convert to DataFrame
    df = pd.DataFrame(processed_data)

    # Filter only rows with size columns (e.g., "M," "L," "XL")
    size_columns = ["M", "L", "XL"]
    relevant_columns = [col for col in df.columns if col in size_columns or col not in size_columns]

    # Reorganize DataFrame to match expected output structure
    df = df[relevant_columns]

    return df

def save_to_excel_formatted(df, output_file):
    # Save the DataFrame to an Excel file with formatting
    with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
        df.to_excel(writer, index=False, sheet_name="Sheet1")
        worksheet = writer.sheets["Sheet1"]

        # Set column widths for better readability
        for i, col in enumerate(df.columns):
            column_width = max(df[col].astype(str).map(len).max(), len(col))
            worksheet.set_column(i, i, column_width)

    print(f"Formatted data successfully saved to {output_file}")

def main():
    pdf_path = r"E:\git\Automated-Email-Parsing-and-Document-Generation\input_doc.pdf"  # Replace with your PDF path
    dimensions, headers = extract_dimensions_with_sizes(pdf_path)

    # Process dimensions and headers dynamically
    formatted_df = process_dimensions_with_sizes(dimensions, headers)

    # Save to Excel
    output_file = "final_output.xlsx"
    save_to_excel_formatted(formatted_df, output_file)

if __name__ == "__main__":
    main()


Formatted data successfully saved to final_output.xlsx
